In [1]:
from google.colab import drive
drive.mount("/content/drive")
DRIVE_SRC = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/experiments/src"
!rm -r -f ./sample_data
!cp -r /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/.creds/.aws ~/
!cp -r $DRIVE_SRC ./src

KeyboardInterrupt: ignored

In [ ]:
! pip install -qq sagemaker ultralytics

In [ ]:
project_name = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs"

# Batsmen Segmentation

In [ ]:
experiment_name = "batsmen-segmentation"

In [ ]:
from src.data.download import download
ds_config_path = download(experiment_name)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-seg.pt')

patience = 20
epochs = 200

model.train(data=ds_config_path, epochs=epochs, patience=patience, pretrained=True, project=project_name, name=experiment_name)

In [ ]:
# !rm -r /content/datasets

# Wicket Classification

In [ ]:
experiment_name = "wicket-classification"

In [ ]:
from src.data.download import download
ds_config_path = download(experiment_name)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-cls.pt')

patience = 10
epochs = 200

model.train(data=experiment_name, epochs=epochs, patience=patience, pretrained=True, project=project_name, name=experiment_name, imgsz=256)

In [ ]:
# !rm -r /content/datasets

# Cricket Object Detection

In [ ]:
experiment_name = "cricket-object-detect"

In [ ]:
from src.data.download import download
config_path = download(experiment_name)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

patience = 10
epochs = 200
model.train(data=config_path, epochs=epochs, patience=patience, project_name=project_name, experiment_name=experiment_name)

In [ ]:
# !rm -r /content/datasets